### Predictive modeling of Orders (Order_payment, review, items and Orders)


In [7]:
import pandas as pd
import snowflake.connector
import os
from dotenv import load_dotenv
from pathlib import Path
# Carrega as variáveis de ambiente
load_dotenv(dotenv_path=Path('.') / 'environment.env')
SF_USER = os.getenv("SF_USER")
SF_PASSWORD = os.getenv("SF_PASSWORD")
SF_ACCOUNT = os.getenv("SF_ACCOUNT")
SF_WAREHOUSE = os.getenv("SF_WAREHOUSE")
SF_DATABASE = os.getenv("SF_DATABASE")
SF_SCHEMA = os.getenv("SF_SCHEMA")

# Conecta ao Snowflake
conn = snowflake.connector.connect(
    user=SF_USER,
    password=SF_PASSWORD,
    account=SF_ACCOUNT,
    warehouse=SF_WAREHOUSE,
    database=SF_DATABASE,
    schema=SF_SCHEMA
)

# Carrega as tabelas
print("Carregando tabelas do Snowflake...")
df_orders = pd.read_sql("SELECT * FROM orders_refined", conn)
df_orders_reviews = pd.read_sql("SELECT * FROM order_reviews_refined", conn)
df_order_payments = pd.read_sql("SELECT * FROM order_payments_refined", conn)
df_order_items = pd.read_sql("SELECT * FROM order_items_refined", conn)
# 💡 NOVO: Carregando a tabela de produtos
df_products = pd.read_sql("SELECT * FROM products_refined", conn)

# Fecha a conexão
conn.close()

# Padroniza os nomes das colunas
for df in [df_orders, df_orders_reviews, df_order_payments, df_order_items, df_products]:
    df.columns = df.columns.str.lower()

# Realiza as junções sequenciais
print("Unindo as tabelas...")
df_full_orders = df_orders.merge(df_orders_reviews, on='order_id', how='left')
df_full_orders = df_full_orders.merge(df_order_payments, on='order_id', how='left')
df_full_orders = df_full_orders.merge(df_order_items, on='order_id', how='left')
# 💡 NOVO: Juntando a tabela de produtos
df_full_orders = df_full_orders.merge(df_products, on='product_id', how='left')

print("Junção completa. O DataFrame final tem o formato:", df_full_orders.shape)
print(df_full_orders.head())

Carregando tabelas do Snowflake...


C:\Users\pacie\AppData\Local\Temp\ipykernel_26324\3396013731.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_orders = pd.read_sql("SELECT * FROM orders_refined", conn)
C:\Users\pacie\AppData\Local\Temp\ipykernel_26324\3396013731.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_orders_reviews = pd.read_sql("SELECT * FROM order_reviews_refined", conn)
C:\Users\pacie\AppData\Local\Temp\ipykernel_26324\3396013731.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_order_payments = pd.read_sql("SELECT * FROM o

Unindo as tabelas...
Junção completa. O DataFrame final tem o formato: (119143, 54)
                           order_id                       customer_id  \
0  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
1  85ce859fd6dc634de8d2f1e290444043  059f7fc5719c7da6cbafe370971a8d70   
2  6ea2f835b4556291ffdc53fa0b3b95e8  c7340080e394356141681bd4c9b8fe31   
3  68873cf91053cd11e6b49a766db5af1a  4632eb5a8f175f6fe020520ae0c678f3   
4  8f06cc6465925031568537b815f1198d  9916715c2ab6ee1710c9c32f0a534ad2   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      1511033286000000000  1511034359000000000   
1    delivered      1511222621000000000  1511223262000000000   
2    delivered      1511558868000000000  1511569269000000000   
3    delivered      1512079335000000000  1512183078000000000   
4    delivered      1510745501000000000  1510746402000000000   

  order_delivered_carrier_date order_delivered_customer_date  \
0          15113579990000000

### Treinamento 

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# --- 1. Preparação dos Dados ---
print("Iniciando a preparação dos dados...")

# Limpeza da variável alvo e das features
df_full_orders['review_score'] = pd.to_numeric(df_full_orders['review_score'], errors='coerce')
df_full_orders['delivery_delay_hours'] = pd.to_numeric(df_full_orders['delivery_delay_hours'], errors='coerce')

# 💡 CORREÇÃO: Converte todas as features para tipo numérico ANTES de calcular a mediana
numeric_features = [
    'price',
    'freight_value',
    'payment_installments',
    'total_delivery_time_hours',
    'shipping_time_hours',
    'product_weight_g',
    'product_volume_cm3'
]
# Ajuste das features de acordo com o seu código anterior. Remova os comentários se quiser usá-las.
# numeric_features = [
#     'price',
#     'freight_value',
#     'payment_installments',
#     'total_delivery_time_hours',
# ]

for col in numeric_features:
    df_full_orders[col] = pd.to_numeric(df_full_orders[col], errors='coerce')

# Preenche os valores nulos (NaN) com a mediana, uma abordagem comum para evitar distorções
for col in numeric_features:
    median_val = df_full_orders[col].median()
    df_full_orders[col].fillna(median_val, inplace=True)

# Remove as linhas onde a variável alvo é nula
df_full_orders.dropna(subset=['review_score', 'delivery_delay_hours'], inplace=True)


# --- 2. Modelagem para Prever 'is_satisfied' (Classificação) ---
print("\n--- Modelagem para prever se o cliente está satisfeito ---")

# 💡 NOVO: Cria a coluna 'is_satisfied'
df_full_orders['is_satisfied'] = df_full_orders['review_score'].apply(lambda score: 1 if score >= 4 else 0)

# Define as features (X) e a variável alvo (y)
X = df_full_orders[numeric_features]
y_satisfied = df_full_orders['is_satisfied']

# Divide os dados em treino e teste (test_size 0.25 para este exemplo)
X_train, X_test, y_train, y_test = train_test_split(
    X, y_satisfied, test_size=0.25, random_state=42
)

# 💡 NOVO: Modelos de Classificação e Métricas
models_classification = {
    "Logistic Regression": LogisticRegression(random_state=42),
    "Random Forest Classifier": RandomForestClassifier(n_estimators=10, random_state=42),
    "XGBoost Classifier": XGBClassifier(n_estimators=10, random_state=42)
}

for name, model in models_classification.items():
    print(f"\nTreinando {name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print(f"Métricas para {name}:")
    print(f"Acurácia: {accuracy:.4f}")
    print(f"Precisão: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")


# --- 3. Modelagem para Prever 'delivery_delay_hours' (Regressão) ---
print("\n--- Previsão de Delivery Delay Hours ---")

# Define as features (X) e a nova variável alvo (y)
X_delay = df_full_orders[numeric_features]
y_delay = df_full_orders['delivery_delay_hours']

# Divide os dados em treino e teste
X_train_delay, X_test_delay, y_train_delay, y_test_delay = train_test_split(
    X_delay, y_delay, test_size=0.25, random_state=42
)

# 💡 Modelos de Regressão e Métricas (Mantidos)
models_regression = {
    "Linear Regression": LinearRegression(),
    "Random Forest Regressor": RandomForestRegressor(n_estimators=10, random_state=42),
    "XGBoost Regressor": XGBRegressor(n_estimators=10, random_state=42)
}

for name, model in models_regression.items():
    print(f"\nTreinando {name}...")
    model.fit(X_train_delay, y_train_delay)
    y_pred_delay = model.predict(X_test_delay)

    r2 = r2_score(y_test_delay, y_pred_delay)
    mae = mean_absolute_error(y_test_delay, y_pred_delay)
    rmse = np.sqrt(mean_squared_error(y_test_delay, y_pred_delay))

    print(f"Métricas para {name}:")
    print(f"R²: {r2:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")

Iniciando a preparação dos dados...

--- Modelagem para prever se o cliente está satisfeito ---

Treinando Logistic Regression...


C:\Users\pacie\AppData\Local\Temp\ipykernel_26324\3437328419.py:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_full_orders[col].fillna(median_val, inplace=True)
C:\Users\pacie\AppData\Local\Temp\ipykernel_26324\3437328419.py:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

F

Métricas para Logistic Regression:
Acurácia: 0.7748
Precisão: 0.7795
Recall: 0.9840
F1-Score: 0.8699

Treinando Random Forest Classifier...
Métricas para Random Forest Classifier:
Acurácia: 0.8201
Precisão: 0.8477
Recall: 0.9324
F1-Score: 0.8880

Treinando XGBoost Classifier...
Métricas para XGBoost Classifier:
Acurácia: 0.7911
Precisão: 0.7951
Recall: 0.9793
F1-Score: 0.8777

--- Previsão de Delivery Delay Hours ---

Treinando Linear Regression...
Métricas para Linear Regression:
R²: 0.3978
MAE: 138.5850
RMSE: 187.6531

Treinando Random Forest Regressor...
Métricas para Random Forest Regressor:
R²: 0.6032
MAE: 105.7416
RMSE: 152.3176

Treinando XGBoost Regressor...
Métricas para XGBoost Regressor:
R²: 0.5015
MAE: 121.1241
RMSE: 170.7282
